# 파이썬 DB 비교 : MySQL(PyMySQL) vs Oracle(python-oracledb) vs SQLite3


## 1. 공통 준비 사항

- **설치 라이브러리**
  - MySQL: `pip install pymysql`
  - Oracle: `pip install oracledb`
  - SQLite3: (파이썬 표준 라이브러리, 설치 불필요)



## 2. 세 가지 DB 비교표

| 구분 | MySQL (PyMySQL) | Oracle (python-oracledb) | SQLite3 |
|------|-----------------|--------------------------|---------|
| 기본 포트 | 3306 | 1521 | 없음 (파일 기반) |
| 데이터 타입 | `VARCHAR`, `INT`, `DATETIME` | `VARCHAR2`, `NUMBER`, `DATE` | `TEXT`, `INTEGER`, `REAL` |
| 자동 증가 키 | `AUTO_INCREMENT` | `IDENTITY` 또는 시퀀스 필요 | `AUTOINCREMENT` |
| 파라미터 바인딩 | `%s` | `:1`, `:name` | `?` |
| 장점 | 오픈소스, 웹개발에 적합 | 안정성, 대규모 트랜잭션 지원 | 설치 불필요, 가볍고 이식성 높음 |
| 단점 | 대규모 서비스 성능 한계 | 설치·관리 복잡 | 대규모 동시접속 불리 |


## 3. DB 연결 예제

In [ ]:

# MySQL 연결 (PyMySQL)
import pymysql

conn_mysql = pymysql.connect(
    host="localhost",
    user="root",
    password="1234",
    database="testdb",
    charset="utf8mb4"
)
print("MySQL 연결 성공:", conn_mysql.open)


In [ ]:

# Oracle 연결 (python-oracledb Thin 모드)
import oracledb

conn_oracle = oracledb.connect(
    user="scott",
    password="tiger",
    dsn="localhost/orclpdb1"  # host/service_name 형식
)
print("Oracle 연결 성공:", conn_oracle.version)


In [ ]:

# SQLite3 연결
import sqlite3

conn_sqlite = sqlite3.connect("testdb.sqlite3")
print("SQLite3 연결 성공:", conn_sqlite)


## 4. 테이블 생성 예제

In [ ]:

# MySQL
cursor_mysql = conn_mysql.cursor()
cursor_mysql.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    age INT
)
""")
print("MySQL students 테이블 생성 완료")


In [ ]:

# Oracle
cursor_oracle = conn_oracle.cursor()
cursor_oracle.execute("""
BEGIN
   EXECUTE IMMEDIATE 'DROP TABLE students';
EXCEPTION
   WHEN OTHERS THEN NULL;
END;
""")
cursor_oracle.execute("""
CREATE TABLE students (
    id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    name VARCHAR2(50),
    age NUMBER
)
""")
print("Oracle students 테이블 생성 완료")


In [ ]:

# SQLite3
cursor_sqlite = conn_sqlite.cursor()
cursor_sqlite.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    age INTEGER
)
""")
print("SQLite3 students 테이블 생성 완료")


## 5. 데이터 삽입 예제

In [ ]:

# MySQL (%s placeholder)
cursor_mysql.execute("INSERT INTO students (name, age) VALUES (%s, %s)", ("홍길동", 20))
conn_mysql.commit()


In [ ]:

# Oracle (:1, :2 placeholder)
cursor_oracle.execute("INSERT INTO students (name, age) VALUES (:1, :2)", ("홍길동", 20))
conn_oracle.commit()


In [ ]:

# SQLite3 (? placeholder)
cursor_sqlite.execute("INSERT INTO students (name, age) VALUES (?, ?)", ("홍길동", 20))
conn_sqlite.commit()


## 6. 데이터 조회 예제

In [ ]:

cursor_mysql.execute("SELECT * FROM students")
print("MySQL:", cursor_mysql.fetchall())

cursor_oracle.execute("SELECT * FROM students")
print("Oracle:", cursor_oracle.fetchall())

cursor_sqlite.execute("SELECT * FROM students")
print("SQLite3:", cursor_sqlite.fetchall())



## 7. 정리

- **MySQL (PyMySQL)** → 오픈소스, 웹서비스용으로 널리 쓰임  
- **Oracle (python-oracledb)** → 대규모 트랜잭션, 엔터프라이즈 환경에 강점  
- **SQLite3** → 가볍고 설치 필요 없음. 실습 및 소규모 앱에 적합  

👉 세 DB 모두 파라미터 바인딩을 지원하지만, 문법이 다르므로 주의해야 함.  
